In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import copy
import os
import pickle
import sys
import numpy as np
import json

np.set_printoptions(precision=3, suppress=True)
# uncomment and replace with correct path if there are import errors
# sys.path.append("/path/to/surface-sampling/")
# sys.path.append("/path/to/NeuralForceField/")
# os.environ["LAMMPS_POTENTIALS"] = "/path/to/lammps/potentials/"
# os.environ["LAMMPS_COMMAND"] ="/path/to/lammps/src/lmp_serial"
# os.environ["ASE_LAMMPSRUN_COMMAND"] = os.environ["LAMMPS_COMMAND"]

import ase

from mcmc import MCMC
from mcmc.system import SurfaceSystem

from time import perf_counter

Initialize test slab and parameters

In [3]:
# Load prepared pristine slab
element = []
slab_pkl = open("../tutorials/data/SrTiO3_001_2x2_pristine_slab.pkl", "rb")
slab = pickle.load(slab_pkl)
offset_data_path = os.path.join(
        "../tutorials",
        "data/nff",
        "offset_data.json",
    )

ref_slab_files = [
    "/mnt/data0/dux/2023_paper_figures/data/final_data/SrTiO3/selected_terms/optim_slab_run_001_O44Sr12Ti16.cif",
    "/mnt/data0/dux/2023_paper_figures/data/final_data/SrTiO3/selected_terms/optim_slab_run_001_O36Sr12Ti12.cif",
    "/mnt/data0/dux/2023_paper_figures/data/final_data/SrTiO3/selected_terms/optim_slab_run_001_O40Sr16Ti12.cif"
]

ref_slabs = [ase.io.read(f) for f in ref_slab_files]

In [4]:
system_settings = {
    "surface_name": 'SrTiO3(001)',
    "cutoff": 5.0,
    "near_reduce": 0.01,
    "planar_distance": 1.55,
    "no_obtuse_hollow": True

}

sampling_settings = {
    "alpha": 1.0, # no annealing
    "temperature": 1.0, # in terms of kbT
    "num_sweeps": 10,
    "sweep_size": 64,
}

calc_settings = {
    "calc_name": "NFF",
    "chem_pots": {"Sr": -2, "Ti": 0, "O": 0},
    "offset_data": json.load(open(offset_data_path, "r")),
    "optimizer": "BFGS",
    "relax_atoms": True,
    "relax_steps": 20,
    "offset": True,
}

Obtain adsorption sites

In [5]:
from pymatgen.analysis.adsorption import AdsorbateSiteFinder
from pymatgen.io.ase import AseAtomsAdaptor

pristine_slab = slab.copy()
pristine_pmg_slab = AseAtomsAdaptor.get_structure(pristine_slab)
site_finder = AdsorbateSiteFinder(pristine_pmg_slab)
# new standardized params
# ads_positions = site_finder.find_adsorption_sites(
#     put_inside=True,
#     symm_reduce=False,
#     near_reduce=system_settings['near_reduce'],
#     distance=system_settings["planar_distance"],
#     no_obtuse_hollow=system_settings['no_obtuse_hollow'],
# )["all"]

# old params
sites = site_finder.find_adsorption_sites(
    distance=1.55, put_inside=False, symm_reduce=False, no_obtuse_hollow=False
)
ads_positions = sites["all"]

print("adsorption coordinates are")
print(ads_positions)

sweep_size = len(ads_positions)

adsorption coordinates are
[array([-0.   , -0.029, 18.87 ]), array([ 1.968,  1.951, 18.829]), array([ 1.968,  0.146, 18.782]), array([-0.   ,  3.956, 18.87 ]), array([ 1.968,  5.936, 18.829]), array([ 1.968,  4.131, 18.782]), array([ 3.936, -0.029, 18.87 ]), array([ 5.903,  1.951, 18.829]), array([ 5.903,  0.146, 18.782]), array([ 3.936,  3.956, 18.87 ]), array([ 5.903,  5.936, 18.829]), array([ 5.903,  4.131, 18.782]), array([ 9.839, 30.936, 18.806]), array([ 8.855, 30.848, 18.85 ]), array([29.517, 30.936, 18.806]), array([30.5  , 30.848, 18.85 ]), array([14.758, 15.998, 18.826]), array([15.742, 13.906, 18.829]), array([30.5  , 28.856, 18.85 ]), array([ 8.855, 28.856, 18.85 ]), array([10.823, 30.848, 18.85 ]), array([14.758, 24.871, 18.85 ]), array([16.726, 24.871, 18.85 ]), array([15.742, 25.861, 18.829]), array([30.5  , 27.953, 18.826]), array([29.517, 28.943, 18.806]), array([28.533, 28.856, 18.85 ]), array([28.533, 27.953, 18.826]), array([12.79 , 15.998, 18.826]), array([12.79 , 

Set up NFF (calculator). We are using neural network weights from our Zenodo dataset (https://zenodo.org/record/7927039). The ensemble requires an `offset_data.json` file

In [6]:
import torch
from nff.io import NeuralFF, AtomsBatch, EnsembleNFF
from mcmc.calculators import EnsembleNFFSurface

if torch.cuda.is_available():
    DEVICE = 0
else:
    DEVICE = "cpu"

# requires an ensemble of models in this path and an `offset_data.json` file
nnids = ["model01", "model02", "model03"]
model_dirs = [
    os.path.join(
        "../tutorials",
        "data/nff",
        str(x),
        "best_model",
    )
    for x in nnids
]

models = []
for modeldir in model_dirs:
    m = NeuralFF.from_file(modeldir, device=DEVICE).model
    models.append(m)

nff_surf_calc = EnsembleNFFSurface(models, device=DEVICE)
nff_surf_calc.set(**calc_settings)

offset data: {'bulk_energies': {'O': -0.17747231201, 'Sr': -0.06043637668, 'SrTiO3': -1.470008697358702}, 'stoidict': {'Sr': 0.49995161381315867, 'Ti': -0.0637500349111578, 'O': -0.31241304903276834, 'offset': -11.324476454433157}, 'stoics': {'Sr': 1, 'Ti': 1, 'O': 3}, 'ref_formula': 'SrTiO3', 'ref_element': 'Ti'} is set from parameters
chemical potentials: {'Sr': -2, 'Ti': 0, 'O': 0} are set from parameters
offset data: {'bulk_energies': {'O': -0.17747231201, 'Sr': -0.06043637668, 'SrTiO3': -1.470008697358702}, 'stoidict': {'Sr': 0.49995161381315867, 'Ti': -0.0637500349111578, 'O': -0.31241304903276834, 'offset': -11.324476454433157}, 'stoics': {'Sr': 1, 'Ti': 1, 'O': 3}, 'ref_formula': 'SrTiO3', 'ref_element': 'Ti'} is set from parameters


Initialize surface system

In [7]:
# set attributes
slab_batch = AtomsBatch(
    positions=slab.positions,
    numbers=slab.numbers,
    cell=slab.cell,
    pbc=True,
    cutoff=system_settings["cutoff"],
    props={"energy": 0, "energy_grad": []},
    calculator=nff_surf_calc,
    requires_large_offsets=True,
    directed=True,
    device=DEVICE,
)

# fix bulk atoms
num_bulk_atoms = len(slab_batch)
bulk_indices = list(range(num_bulk_atoms))
print(f"bulk indices {bulk_indices}")
surf_indices = slab.get_surface_atoms()

fix_indices = list(set(bulk_indices) - set(surf_indices))
print(f"fix indices {fix_indices}")

# bulk atoms are 0 and surface atoms are 1
tags = np.ones(num_bulk_atoms)
tags[fix_indices] = 0
slab_batch.set_tags(tags)
print(f"tags {tags}")

surface = SurfaceSystem(slab_batch, ads_positions, nff_surf_calc, system_settings=system_settings)
surface.all_atoms.write("SrTiO3_001_2x2_all_virtual_ads.cif")

bulk indices [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]
fix indices [0, 1, 2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 24, 25, 26, 27, 28, 30, 31, 32, 33, 34, 35, 36, 39, 40, 41, 42, 43, 45, 46, 47, 48, 49, 50, 51, 54, 55, 56, 57, 58]
tags [0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1.]
ads coords is [-0.    -0.029 18.87 ]
ads coords is [ 1.968  1.951 18.829]
ads coords is [ 1.968  0.146 18.782]
ads coords is [-0.     3.956 18.87 ]
ads coords is [ 1.968  5.936 18.829]
ads coords is [ 1.968  4.131 18.782]
ads coords is [ 3.936 -0.029 18.87 ]
ads coords is [ 5.903  1.951 18.829]
ads coords is [ 5.903  0.146 18.782]
ads coords is [ 3.936  3.956 18.87 ]


2024-02-13 10:09:42,492|INFO|initial state is [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
2024-02-13 10:09:42,492|INFO|number of pristine atoms is 60
2024-02-13 10:09:42,493|INFO|bulk indices are [ 0  1  2  3  4  5  6  9 10 11 12 13 15 16 17 18 19 20 21 24 25 26 27 28
 30 31 32 33 34 35 36 39 40 41 42 43 45 46 47 48 49 50 51 54 55 56 57 58]
2024-02-13 10:09:42,493|INFO|surface indices are [ 7  8 14 22 23 29 37 38 44 52 53 59]


In [8]:
from mcmc.energy import slab_energy
slab_energy(surface, offset=True, offset_data=calc_settings["offset_data"])

      Step     Time          Energy         fmax
BFGS:    0 10:09:42     -467.521851        0.2306
BFGS:    1 10:09:42     -467.527740        0.1610
BFGS:    2 10:09:42     -467.533905        0.1001
BFGS:    3 10:09:42     -467.535370        0.1234
BFGS:    4 10:09:42     -467.539398        0.1376
BFGS:    5 10:09:42     -467.541351        0.0825
BFGS:    6 10:09:42     -467.542206        0.0278
BFGS:    7 10:09:43     -467.542328        0.0162
BFGS:    8 10:09:43     -467.542358        0.0149
BFGS:    9 10:09:43     -467.542358        0.0125
BFGS:   10 10:09:43     -467.542389        0.0158
BFGS:   11 10:09:43     -467.542480        0.0177
BFGS:   12 10:09:43     -467.542511        0.0134
BFGS:   13 10:09:43     -467.542511        0.0088


(12.469390869140625, 0.31367528438568115, 0.0, 0.0, [])

Test out other reference slabs

In [9]:
ref_slab_batches = [AtomsBatch(
    positions=slab.positions,
    numbers=slab.numbers,
    cell=slab.cell,
    pbc=True,
    cutoff=system_settings["cutoff"],
    props={"energy": 0, "energy_grad": []},
    calculator=nff_surf_calc,
    requires_large_offsets=True,
    directed=True,
    device=DEVICE,
) for slab in ref_slabs]


ref_surfs = []
# additional atoms are adsorbates and should be labelled as 2
for ref_slab_batch in ref_slab_batches:
    num_ads = len(ref_slab_batch) - len(tags)
    curr_tags = np.int32(tags.tolist() + [2] * num_ads)
    print(f"tags {curr_tags}")
    ref_slab_batch.set_tags(curr_tags)
    ref_surf = SurfaceSystem(ref_slab_batch, ads_positions, nff_surf_calc, system_settings=system_settings)
    ref_surfs.append(ref_surf)


tags [0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0
 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 2 2 2 2 2 2 2 2 2 2 2 2]


ads coords is [-0.    -0.029 18.87 ]
ads coords is [ 1.968  1.951 18.829]
ads coords is [ 1.968  0.146 18.782]
ads coords is [-0.     3.956 18.87 ]
ads coords is [ 1.968  5.936 18.829]
ads coords is [ 1.968  4.131 18.782]
ads coords is [ 3.936 -0.029 18.87 ]
ads coords is [ 5.903  1.951 18.829]
ads coords is [ 5.903  0.146 18.782]
ads coords is [ 3.936  3.956 18.87 ]
ads coords is [ 5.903  5.936 18.829]
ads coords is [ 5.903  4.131 18.782]
ads coords is [ 9.839 30.936 18.806]
ads coords is [ 8.855 30.848 18.85 ]
ads coords is [29.517 30.936 18.806]
ads coords is [30.5   30.848 18.85 ]
ads coords is [14.758 15.998 18.826]
ads coords is [15.742 13.906 18.829]
ads coords is [30.5   28.856 18.85 ]
ads coords is [ 8.855 28.856 18.85 ]
ads coords is [10.823 30.848 18.85 ]
ads coords is [14.758 24.871 18.85 ]
ads coords is [16.726 24.871 18.85 ]
ads coords is [15.742 25.861 18.829]
ads coords is [30.5   27.953 18.826]
ads coords is [29.517 28.943 18.806]
ads coords is [28.533 28.856 18.85 ]
a

2024-02-13 10:09:44,735|INFO|initial state is [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
2024-02-13 10:09:44,735|INFO|number of pristine atoms is 72
2024-02-13 10:09:44,736|INFO|bulk indices are [ 0  1  2  3  4  5  6  9 10 11 12 13 15 16 17 18 19 20 21 24 25 26 27 28
 30 31 32 33 34 35 36 39 40 41 42 43 45 46 47 48 49 50 51 54 55 56 57 58]
2024-02-13 10:09:44,736|INFO|surface indices are [ 7  8 14 22 23 29 37 38 44 52 53 59]


tags [0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0
 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1]
ads coords is [-0.    -0.029 18.87 ]
ads coords is [ 1.968  1.951 18.829]
ads coords is [ 1.968  0.146 18.782]
ads coords is [-0.     3.956 18.87 ]
ads coords is [ 1.968  5.936 18.829]
ads coords is [ 1.968  4.131 18.782]
ads coords is [ 3.936 -0.029 18.87 ]
ads coords is [ 5.903  1.951 18.829]
ads coords is [ 5.903  0.146 18.782]
ads coords is [ 3.936  3.956 18.87 ]
ads coords is [ 5.903  5.936 18.829]
ads coords is [ 5.903  4.131 18.782]
ads coords is [ 9.839 30.936 18.806]
ads coords is [ 8.855 30.848 18.85 ]
ads coords is [29.517 30.936 18.806]
ads coords is [30.5   30.848 18.85 ]
ads coords is [14.758 15.998 18.826]
ads coords is [15.742 13.906 18.829]
ads coords is [30.5   28.856 18.85 ]
ads coords is [ 8.855 28.856 18.85 ]
ads coords is [10.823 30.848 18.85 ]
ads coords is [14.758 24.871 18.85 ]
ads coords is [16.726 24.871 18.85 ]
ads coords is [15.742

2024-02-13 10:09:45,982|INFO|initial state is [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
2024-02-13 10:09:45,983|INFO|number of pristine atoms is 60
2024-02-13 10:09:45,983|INFO|bulk indices are [ 0  1  2  3  4  5  6  9 10 11 12 13 15 16 17 18 19 20 21 24 25 26 27 28
 30 31 32 33 34 35 36 39 40 41 42 43 45 46 47 48 49 50 51 54 55 56 57 58]
2024-02-13 10:09:45,984|INFO|surface indices are [ 7  8 14 22 23 29 37 38 44 52 53 59]


tags [0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0
 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 2 2 2 2 2 2 2 2]
ads coords is [-0.    -0.029 18.87 ]
ads coords is [ 1.968  1.951 18.829]
ads coords is [ 1.968  0.146 18.782]
ads coords is [-0.     3.956 18.87 ]
ads coords is [ 1.968  5.936 18.829]
ads coords is [ 1.968  4.131 18.782]
ads coords is [ 3.936 -0.029 18.87 ]
ads coords is [ 5.903  1.951 18.829]
ads coords is [ 5.903  0.146 18.782]
ads coords is [ 3.936  3.956 18.87 ]
ads coords is [ 5.903  5.936 18.829]
ads coords is [ 5.903  4.131 18.782]
ads coords is [ 9.839 30.936 18.806]
ads coords is [ 8.855 30.848 18.85 ]
ads coords is [29.517 30.936 18.806]
ads coords is [30.5   30.848 18.85 ]
ads coords is [14.758 15.998 18.826]
ads coords is [15.742 13.906 18.829]
ads coords is [30.5   28.856 18.85 ]
ads coords is [ 8.855 28.856 18.85 ]
ads coords is [10.823 30.848 18.85 ]
ads coords is [14.758 24.871 18.85 ]
ads coords is [16.726 24.871 18.85 ]
ads c

2024-02-13 10:09:47,228|INFO|initial state is [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
2024-02-13 10:09:47,228|INFO|number of pristine atoms is 68
2024-02-13 10:09:47,229|INFO|bulk indices are [ 0  1  2  3  4  5  6  9 10 11 12 13 15 16 17 18 19 20 21 24 25 26 27 28
 30 31 32 33 34 35 36 39 40 41 42 43 45 46 47 48 49 50 51 54 55 56 57 58]
2024-02-13 10:09:47,230|INFO|surface indices are [ 7  8 14 22 23 29 37 38 44 52 53 59]


In [10]:
for surf in ref_surfs:
    results = slab_energy(surf)
    surf_energy = results[0]
    print(f"energy of reference slab is {surf_energy}")

      Step     Time          Energy         fmax
BFGS:    0 10:09:47     -570.128052        1.3431
BFGS:    1 10:09:47     -570.248047        0.9791
BFGS:    2 10:09:47     -570.414490        0.7114
BFGS:    3 10:09:47     -570.452393        0.5441
BFGS:    4 10:09:47     -570.538391        0.3433
BFGS:    5 10:09:47     -570.551575        0.2750
BFGS:    6 10:09:47     -570.581787        0.2215
BFGS:    7 10:09:47     -570.586609        0.2149
BFGS:    8 10:09:47     -570.598877        0.1553
BFGS:    9 10:09:47     -570.603455        0.1359
BFGS:   10 10:09:47     -570.607666        0.1231
BFGS:   11 10:09:47     -570.611084        0.0945
BFGS:   12 10:09:47     -570.613953        0.0803
BFGS:   13 10:09:47     -570.615662        0.0691
BFGS:   14 10:09:48     -570.616882        0.0669
BFGS:   15 10:09:48     -570.618042        0.0537
BFGS:   16 10:09:48     -570.618958        0.0572
BFGS:   17 10:09:48     -570.619629        0.0467
BFGS:   18 10:09:48     -570.620300        0.0500
B